In [4]:
# Install required libraries
!pip install rasterio fiona numpy

# Import the required libraries
import os
import rasterio
import fiona
from rasterio.features import rasterize
import numpy as np

# Define input folder and output raster path
input_folder = 'dataset'  # Adjust this to your actual path if different
output_raster = 'output_combined_mask_final.tif'

# Define pixel values for each feature as specified
pixel_values = {
    'road_bound': [0, 0, 200],
    'buildings': [255, 0, 0],
    'road_markings': {
        'broken_line': [0, 20, 10],
        'cycle_lane': [0, 40, 0],
        'dashed_line': [0, 45, 70],
        'pedestrian_crossing': [0, 100, 0],
        'solid_line': [0, 45, 0],
        'stop_line': [0, 85, 0]
    },
    'null': [180, 180, 180]  # Light gray color for null values
}

def check_shapefile_components(shapefile_path):
    """
    Check if all components of a shapefile are present.
    
    Args:
    shapefile_path (str): Path to the .shp file of the shapefile.
    
    Raises:
    FileNotFoundError: If any of the shapefile components are missing.
    """
    base_path = os.path.splitext(shapefile_path)[0]
    required_extensions = ['.shp', '.shx', '.dbf', '.prj']
    
    for ext in required_extensions:
        component_path = base_path + ext
        if not os.path.exists(component_path):
            raise FileNotFoundError(f"Shapefile component not found: {component_path}")

def rasterize_vector(input_folder, output_raster, pixel_values):
    """
    Rasterize vector data into a single raster mask with specified pixel values.
    
    Args:
    input_folder (str): Folder containing the input data.
    output_raster (str): Path to save the output raster file.
    pixel_values (dict): Dictionary containing pixel values for different features.
    """
    raster_path = os.path.join(input_folder, 'raster_data.tif')  # Replace with actual raster file name
    
    if not os.path.exists(raster_path):
        raise FileNotFoundError(f"Raster file not found: {raster_path}")
    
    # Open the base raster to get dimensions and geotransformation
    with rasterio.open(raster_path) as src:
        meta = src.meta.copy()
        cols, rows = src.width, src.height
        transform = src.transform
    
    # Create a new raster to store the mask
    meta.update(count=3, dtype=rasterio.uint8)
    
    # Initialize an array to store pixel values
    raster_data = np.zeros((rows, cols, 3), dtype=np.uint8)
    
    # Function to burn vector features into the raster
    def burn_features(shapefile, value):
        shapefile_path = os.path.join(input_folder, shapefile)
        check_shapefile_components(shapefile_path)
        
        with fiona.open(shapefile_path) as shp:
            print(f"Rasterizing {shapefile} with value {value}")
            shapes = [(feature['geometry'], 1) for feature in shp]  # Using 1 as a dummy value to burn
            return rasterize(shapes, out_shape=(rows, cols), transform=transform)
    
    # Rasterize road bound
    road_bound = burn_features('road_bound.shp', pixel_values['road_bound'])
    raster_data[:, :, 0] += road_bound * pixel_values['road_bound'][0]
    raster_data[:, :, 1] += road_bound * pixel_values['road_bound'][1]
    raster_data[:, :, 2] += road_bound * pixel_values['road_bound'][2]
    
    # Check and print road bound raster
    print("Road Bound Raster: ", np.unique(road_bound, return_counts=True))
    
    # Rasterize buildings
    buildings = burn_features('buildings.shp', pixel_values['buildings'])
    raster_data[:, :, 0] = np.maximum(raster_data[:, :, 0], buildings * pixel_values['buildings'][0])
    raster_data[:, :, 1] = np.maximum(raster_data[:, :, 1], buildings * pixel_values['buildings'][1])
    raster_data[:, :, 2] = np.maximum(raster_data[:, :, 2], buildings * pixel_values['buildings'][2])
    
    # Check and print buildings raster
    print("Buildings Raster: ", np.unique(buildings, return_counts=True))
    
    # Rasterize road markings
    for marking_type, color_values in pixel_values['road_markings'].items():
        road_markings = burn_features('road_markings.shp', color_values)
        raster_data[:, :, 0] = np.maximum(raster_data[:, :, 0], road_markings * color_values[0])
        raster_data[:, :, 1] = np.maximum(raster_data[:, :, 1], road_markings * color_values[1])
        raster_data[:, :, 2] = np.maximum(raster_data[:, :, 2], road_markings * color_values[2])
        
        # Check and print road markings raster
        print(f"Road Markings Raster ({marking_type}): ", np.unique(road_markings, return_counts=True))
    
    # Handle null values (black areas)
    null_mask = (raster_data[:, :, 0] == 0) & (raster_data[:, :, 1] == 0) & (raster_data[:, :, 2] == 0)
    raster_data[null_mask] = pixel_values['null']
    
    # Write raster data to the output raster
    with rasterio.open(output_raster, 'w', **meta) as dst:
        dst.write(raster_data[:, :, 0], 1)
        dst.write(raster_data[:, :, 1], 2)
        dst.write(raster_data[:, :, 2], 3)

# Call the function to rasterize vector data into a single raster mask
rasterize_vector(input_folder, output_raster, pixel_values)

print(f"Raster mask successfully created: {output_raster}")

Rasterizing road_bound.shp with value [0, 0, 200]
Road Bound Raster:  (array([0, 1], dtype=uint8), array([194618,  67526]))
Rasterizing buildings.shp with value [255, 0, 0]
Buildings Raster:  (array([0, 1], dtype=uint8), array([222116,  40028]))
Rasterizing road_markings.shp with value [0, 20, 10]
Road Markings Raster (broken_line):  (array([0, 1], dtype=uint8), array([254264,   7880]))
Rasterizing road_markings.shp with value [0, 40, 0]
Road Markings Raster (cycle_lane):  (array([0, 1], dtype=uint8), array([254264,   7880]))
Rasterizing road_markings.shp with value [0, 45, 70]
Road Markings Raster (dashed_line):  (array([0, 1], dtype=uint8), array([254264,   7880]))
Rasterizing road_markings.shp with value [0, 100, 0]
Road Markings Raster (pedestrian_crossing):  (array([0, 1], dtype=uint8), array([254264,   7880]))
Rasterizing road_markings.shp with value [0, 45, 0]
Road Markings Raster (solid_line):  (array([0, 1], dtype=uint8), array([254264,   7880]))
Rasterizing road_markings.shp 